# Beispiel für MS Access / Python / SQLite und Pandas

Zunächst müssen in der Python-Umgebung die richtigen Bibliotheken installiert werden. Dazu benutzen wir die Datei [requirements.txt](requirements.txt) und installieren alle Packages die darin genannt werden.

In [ ]:
!pip install  --no-input -r requirements.txt

### Bibliotheken importieren

Anschliessend importieren wir diese Libraries

In [153]:
import pandas # Allgemeines Arbeiten mit Tabellen, wie ein Data Frame in R
import pyodbc # Datenbanktreiber für MS Access
import sqlalchemy as sa # Hilfsbibliothek für SQL, wird benötigt um MS Access kompatibel mit pandas zu bekommen
import sqlite3 # Datenbanktreiber für SQLite
import openpyxl as excel # Bibliothek die vollen Lese- und Schreibzugriff auf Excel Tabellen ermöglicht
import os # Standardbibliothek in Python, z.B. für Dateioperationen

Wir haben eine Beispiel **MS-Access Datenbank** unter [TestDatabase.accdb](TestDatabase.accdb). Die laden wir jetzt mal, und schauen sie uns an. Dazu definieren wir aber erstmal eine Hilfsfunktion, die erlaubt uns mit einer MS-Access Datenbank zu verbinden.

### Mit MS-Access Datenbank verbinden


In [86]:
def ms_access_connection(db_file):
    import os
    db_abspath = os.path.abspath(db_file)
    print(f"Opening {db_abspath}")
    connection_string = (
        r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
        r"DBQ=" + db_abspath + ";"
        r"ExtendedAnsiSQL=1;"
    )
    connection_url = sa.engine.URL.create(
        "access+pyodbc",
        query={"odbc_connect": connection_string}
    )
    #msa = pyodbc.connect( db_connection_string )
    msa = sqlalchemy.create_engine(connection_url)
    return msa

Jetzt verbinden wir uns mit der MS-Access Datenbank und nennen die Verbindung **db1**

In [88]:
db1 = ms_access_connection("TestDatabase.accdb")

Opening C:\Users\VW6Z1LW\test\TestDatabase.accdb


### Lesen und Schreiben von MS-Access Datenbanken

Jetzt können wir Tabellen aus der MS Access Datenbank auslesen (und ggf. auch reinschreiben)

In [67]:
example_table_data = pandas.read_sql("SELECT * FROM example_table", db1)
example_table_data

,ID,my_json,name,age
0,1,"{ ""a"" : [1,2,3] }",guido,120
1,2,"{""a"" : [4,5,3] }",bert,67


### Kopieren in eine SQLite Datenbank

Um besser mit den Daten arbeitebn zu können, kopieren wir sie in eine extra dafür angelegte SQLIte Datenbank.
Die erzeugen wir hier, und nennen sie **db2**

In [30]:
db2 = sqlite3.connect("meine_datenbank.sqlite")

Jetzt können wir die Tabellendaten die wir vorhin aus MS Access gelesen haben, in die SQLite Datenbank kopieren:

In [77]:
example_table_data.to_sql("example_table", db2, if_exists='replace')

2

### Benutzerdefinierte Funktionen für SQLIte mit Python

Das schöne an sqlite3 ist, dass wir ganz einfach Funktionen in Python schreiben können, die wir dann in Datenbankabfragen verwenden können. Diese Funktion z.B. parst einen JSON String, interpretiert ihn als kleine Tabelle, und gibt den Wert einer Zelle zurück, die anhand von Spaltenname und Zeilenindex gefunden wurde. ( Falls nichts gefunden werden konnte, wird nichts zurückgegeben)

In [72]:
import json # Für dieses Beispiel brauchen wir die JSON Bibliothek, die ist bei Python immer dabei

def get_field(js, column, index):
    try:
        ds = json.loads(js) # Einen String parsen. Damit wird dann aus sowas wie "{'a' : [1,2,3]}" ein Dictionary-Objekt
        if column in ds:
            col = ds[column]
            if len(col)>index:
                return col[index]
        return None
    except:
        return None

Jetzt können wir diese Funktion als Datenbankfunktion mit SQLite registrieren. Sie bekommt den Namen "get_field", und hat 3 Parameter, das müssen wir hier mit angeben.

In [154]:
db2.create_function('get_field', 3, get_field)

Jetzt kann sie in SQL Abfragen verwendet werden.

In [78]:
pandas.read_sql("""
    SELECT name, get_field(my_json, "a", 1) as interesting_field, age FROM example_table
""", db2)

,name,interesting_field,age
0,guido,2,120
1,bert,5,67


Machen wir mal eine kleine Mini-Auswertung mit ihrer Hilfe

In [80]:
auswertung = pandas.read_sql("""
    SELECT name, get_field(my_json, "a", 1) as interesting_field, age from example_table where interesting_field>2
""", db2)
auswertung

,name,interesting_field,age
0,bert,5,67


### Auswertungsergebnisse in MS-Access speichern

Und speichern die Ergebnisse dieser Auswertung als neue Tabelle in der MS Access Datenbank:

In [81]:
auswertung.to_sql("auswertung_aus_python", db1)

-1

In [82]:
pandas.read_sql("SELECT * FROM auswertung_aus_python", db1)

,index,name,interesting_field,age
0,0,bert,5,67


## Arbeiten mit Excel-Tabellen

Das war es erstmal. Jetzt schauen wir, wie man kombinierte oder gesplittete Zellen aus Excel Tabellen in den Griff bekommt, mit Hilfe der openpyxl Library. Die haben wir unter dem Namen "excel" importiert. Die Doku dazu findet man hier: [Openpyxl Dokumentation](https://openpyxl.readthedocs.io/en/stable/)

Neben Openpyxel, hat **pandas** allerdings auch sehr gute Funktionen um Excel-Tabellen zu lesen und zu schreiben. Openpyxel braucht man vor allem, wenn man fortgeschrittene Features braucht ( z.B. Zellen verbinden möchte, Formatierungen ändern will, Formeln anpassen oder ähnliches ). 

Pandas hingegen betrachtet Excel-Tabellen einfach als eine Sammlung von simplen Datentabellen.

Generell gilt: 

 * **pandas** verwenden wir meistens, um Daten aus Excel Tabellen zu lesen und zu schreiben.
 * **openpyxl** verwenden wir meistens, um Excel Tabellen programmatisch umzuformatieren oder komplexere Excel-Tabellen auszulesen.
 
 
 *Nicht vergessen*: Im folgenden benutzen wir "excel" als alias für "openpyxl", weil es einfacher zu schreiben und leichter zu merken ist.

In [155]:
# Laden des ersten Worksheets einer Excel Tabelle mit Hilfe von Pandas
pandas.read_excel("Mappe1.xlsx")

,Name,Alter,Info,Counter
0,Kurt,39.0,was,23
1,NaN,NaN,auch,23
2,NaN,NaN,immer,42
3,Bert,239.0,wie,55
4,NaN,NaN,dem,110
5,NaN,NaN,auch,220
6,NaN,NaN,sei,440


In [114]:
# Laden wir die gleiche Tabellen mit Openpyxl
workbook = excel.open("Mappe1.xlsx")

In [156]:
# Schauen wir uns ein bestimmtes Arbeitsblatt in Excel an:
sheet = workbook["Tabelle1"]

In [157]:
# Und greifen wir auf eine bestimmte Zelle zu ( wir könnten sie auch ändern )
sheet['A2'].value

'Kurt'

In [161]:
# Ändern wir sie mal
sheet['A2'].value = 'Herbert'

In [162]:
# Wie man sieht, werden auch Formeln als solche angezeigt wenn man mit Openpyxl arbeitet.
list(sheet.values)

[('Name', 'Alter', 'Info', 'Counter'),
 ('Herbert', 39, 'was', 23),
 (None, None, 'auch', 23),
 (None, None, 'immer', 42),
 ('Bert', 239, 'wie', 55),
 (None, None, 'dem', '=D5*2'),
 (None, None, 'auch', '=D6*2'),
 (None, None, 'sei', '=D7*2')]

In [122]:
cell = sheet['D7']
cell.value

'=D6*2'

### Nur die Daten aus Excel Tabellen laden ( statt Formeln )

In [163]:
# Wenn wir nicht mit den Formeln, sondern nur mit den Werten arbeiten wollen, müssen wir beim laden "data_only=True"
# angeben, dann werden die letzten von Excel berechneten Werte benutzt statt der Formeln.
# Aber Vorsicht: Wenn man dann speichert, sind die Formeln auch wirklich weg.
workbook2 = excel.open("Mappe1.xlsx", data_only=True)
sheet2 = workbook2["Tabelle1"]

In [164]:
# Jetzt sind in sheet2 nur die berechneten Werte, nicht die Formeln
cell2 = sheet2['D7']
cell2.value

220

#### Zellen programmatisch verschmelzen oder wieder trennen

In [165]:
# Man kann mit Openpyxl auch Zellen programmatisch verschmelzen
sheet2.merge_cells("A2:A4")
sheet2.merge_cells("A5:A8")


In [ ]:
# In diesem Workbook sind die Zellen jetzt verschmolzen, gerne in Excel anschauen...
workbook2.save("Mappe_merged.xlsx")

In [167]:
# Man kann das verschmelzen auch Rückgängig machen
sheet2.unmerge_cells("A2:A4")
sheet2.unmerge_cells("A5:A8")

### Verschmolzene Zellen auffüllen aus den vorhergehenden Zeilen


In [188]:
# Wir brauchen dazu eine Tabelle mit verschmolzenen Zellen, also laden wir die
df_merged = pandas.read_excel("Mappe_merged.xlsx")

In [185]:
df_merged

,Name,Alter,Info,Counter
0,Kurt,39.0,was,23
1,NaN,NaN,auch,23
2,NaN,NaN,immer,42
3,Bert,239.0,wie,55
4,NaN,NaN,dem,110
5,NaN,NaN,auch,220
6,NaN,NaN,sei,440


In [189]:
# Jetzt ersetzen wir fehlende Werte einfach aus der vorherigen Zeile
df_expanded = df_merged.fillna(method='ffill')
df_expanded

,Name,Alter,Info,Counter
0,Kurt,39.0,was,23
1,Kurt,39.0,auch,23
2,Kurt,39.0,immer,42
3,Bert,239.0,wie,55
4,Bert,239.0,dem,110
5,Bert,239.0,auch,220
6,Bert,239.0,sei,440


In [191]:
# Und das ganze können wir jetzt auch wieder speichern. Vielleicht diesmal, ohne den Index:
df_expanded.to_excel("Mappe_merged_expanded.xlsx", index=False)

### Laden eines anderen als des ersten Arbeitsblatts aus einer Excel Tabelle

In [193]:
# Mit dem Parameter "sheet_name" können wir angeben, welches Arbeitsblatt wir lesen wollen
pandas.read_excel("Mappe_merged_expanded.xlsx", sheet_name="Sheet1")

,Name,Alter,Info,Counter
0,Kurt,39,was,23
1,Kurt,39,auch,23
2,Kurt,39,immer,42
3,Bert,239,wie,55
4,Bert,239,dem,110
5,Bert,239,auch,220
6,Bert,239,sei,440


### Auswertungen / Gruppieren mit Hilfe von Pandas

In [194]:
# Gruppieren nach Name und Alter, 
# dann für jede eindeutige Kombination von Name und Alter den Mindestwert von Counter bestimmen
df_expanded.groupby(["Name", "Alter"], as_index=False).Counter.min()

,Name,Alter,Counter
0,Bert,239.0,55
1,Kurt,39.0,23


### Kombinierte Auswertungen / Excel, Datenbanken, Pandas und mehr ..

In [195]:
# Oder das Maximum
df_expanded.groupby(["Name", "Alter"], as_index=False).Counter.max()

,Name,Alter,Counter
0,Bert,239.0,440
1,Kurt,39.0,42


In [199]:
# So eine Auswertung kann man auch einfach in einer MS-Access Tabelle speichern
df_expanded.groupby(["Name", "Alter"], as_index=False).Counter.min().to_sql("max_counter_auswertung", db1,
                                                                            if_exists="replace")

-1

In [202]:
# Man kann natürlch auch die Tabelle die wir vorhin in Excel hatten, jetzt in MS-Access hineinkopieren..
df_expanded.to_sql("df_expanded", db1, if_exists="replace")

-1